
This script takes a product name and a directory, and checks for newly available imagery or gaps in the archive. Imagery is then downloaded and written to disk to complete.


---


Original author: F. Dan O'Neill

In [ ]:
!pip install git+https://github.com/ritviksahajpal/octvi

In [ ]:
import logging, os
logging.basicConfig(level=os.environ.get("LOGLEVEL","INFO"))
log = logging.getLogger(__name__)

import glob, octvi, shutil, sys
from osgeo import gdal
from osgeo.gdal_array import *
import numpy as np
from datetime import datetime, timedelta


def generateFileName(product:str,vi:str, year:int=None,doy:int=None,date:str=None) -> str:
	"""This function converts a date into a valid file name

	Specify EITHER 'year' and 'doy' OR 'date'

	***

	Parameters
	----------
	product:str
		Name of imagery product (e.g. "MOD09CMG")
	year:int
		Year of image
	doy:int
		Day of year (doy) of image
	date:str
		%Y-%m-%d
	"""
	if year is None or doy is None:
		try:
			year,doy = datetime.strptime(date,"%Y-%m-%d").strftime("%Y-%j").split("-")
		except:
			log.error("Specify either year and doy OR date!")

	prefix = product[:5].lower()
	if vi == 'ndvi':
		return f"{prefix}.ndvi.global_0.05_degree.{year}.{str(doy).zfill(3)}.c6.v1.tif"
	elif vi == 'gcvi':
		return f"{prefix}.gcvi.global_0.05_degree.{year}.{str(doy).zfill(3)}.c6.v1.tif"

def check8DayValidity(product:str,date:str) -> bool:
	"""This function returns whether all files in an 8-day range from
	the provided date are available.

	***

	Parameters
	---------
	product:str
		e.g. "MOD09CMG"
	date:str
		%Y-%m-%d
	"""
	for i in range(0,9):
		compDate = (datetime.strptime(date,"%Y-%m-%d") + timedelta(days=i)).strftime("%Y-%m-%d")
		if len(octvi.url.getDates(product,compDate)) == 0:
			return False
	return True

def scaleConversion_glamToMark(in_file:str) -> None:
	"""This function changes the scaling on an input file

	The values are converted from GLAM scaling (NDVI * 10000) to
	Mark's scaling ((NDVI * 200) + 50)

	***

	Parameters
	----------
	in_file:str
		Path to the image file to be converted

	"""
	## define intermediate filename
	extension = os.path.splitext(in_file)[1]
	intermediate_file = in_file.replace(extension,f".TEMP{extension}")
	## copy to intermediate file
	with open(in_file,'rb') as rf:
		with open(intermediate_file,'wb') as wf:
			shutil.copyfileobj(rf,wf)
	## open file with gdal
	ds = gdal.Open(in_file, 0)
	# extract projection and geotransform
	srs = ds.GetProjection()
	gt = ds.GetGeoTransform()
	# convert to array
	ds_band = ds.GetRasterBand(1)
	arr = BandReadAsArray(ds_band)
	rasterYSize, rasterXSize = arr.shape
	# close file and band
	ds = ds_band = None

	## convert array values
	arr = ((arr*.02) + 50)
	arr[arr == -10] = 255
	arr = np.uint8(arr)

	## write to file
	driver = gdal.GetDriverByName('GTiff')
	dataset = driver.Create(in_file,rasterXSize,rasterYSize,1,gdal.GDT_Byte,['COMPRESS=LZW'])
	dataset.GetRasterBand(1).WriteArray(arr)
	dataset.GetRasterBand(1).SetNoDataValue(255)
	dataset.SetGeoTransform(gt)
	dataset.SetProjection(srs)
	dataset.FlushCache() # Write to disk
	del dataset

	## remove intermediate file
	os.remove(intermediate_file)


def download_and_process(directory, start_year:int, print_missing=False):
  from tqdm import tqdm

  runningComposites = glob.glob(os.path.join(directory,"*.running_composite.tif"))
  for c in runningComposites:
    os.remove(c)
  # get list of existing full files
  extantFiles = glob.glob(os.path.join(directory,f"*.tif"))

  ## get missing dates
  # first and last year

  # range of years and doys
  years = [y for y in range(start_year,int(datetime.now().strftime("%Y"))+1)]
  doys =[ d for d in range(1,365,8)]

  # filter missing dates
  dates = {}
  earliestDataDict = {"MOD09CMG":"2000.055","MYD09CMG":"2002.185"}
  for y in tqdm(years, desc='year'):
    for doy in tqdm(doys, desc='doy', leave=False):
      if not os.path.exists(os.path.join(directory,generateFileName("MOD09CMG","ndvi", y,doy))):
        formattedDate = datetime.strptime(f"{y}.{doy}","%Y.%j").strftime("%Y-%m-%d")
        # is the image in the future?
        if datetime.strptime(f"{y}.{doy}","%Y.%j") >= datetime.now() or datetime.strptime(f"{y}.{doy}","%Y.%j") < datetime.strptime(earliestDataDict.get("MOD09CMG","2000.049"),"%Y.%j"):
          continue
        # does the image exist at all?
        if len(octvi.url.getDates("MOD09CMG",formattedDate)) == 0: # if there is valid imagery for the date
          dates[formattedDate] = "No imagery"
        # are there missing files in the compositing period?
        elif not check8DayValidity("MOD09CMG",formattedDate):
          dates[formattedDate] = "Incomplete compositing period"
        # if none of the above, it's available.
        else:
          dates[formattedDate] = "Available"
  # special behavior if "-print_missing" is set
  if print_missing:
    for k in dates.keys():
      outString = f"{k} : {dates[k]}"
      print(outString)
    sys.exit()


  availableFiles = 0
  completedFiles = 0
  for d in dates.keys():
    if dates[d] == "Available":
      availableFiles += 1
      log.info(f"Creating Composite for {d}")
      outPath = os.path.join(directory,generateFileName("MOD09CMG","ndvi",date=d))
      octvi.globalVi("MOD09CMG",d,outPath)
      scaleConversion_glamToMark(outPath)

      ## reproject to WGS 1984
      srs = 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]'
      ds = gdal.Open(outPath,1)
      if ds:
        res = ds.SetProjection(srs)
        if res != 0:
          log.error("--projection failed: {}".format(str(res)))
          ds = None
          continue
        ds = None
        completedFiles += 1
      else:
        log.error("--could not open with GDAL")

      log.info(f"Done. {availableFiles} composites available; {completedFiles} composites created. Use -print_missing flag to see details of missing composites.")


In [ ]:
#Request a key via https://urs.earthdata.nasa.gov/home
octvi.app_key = "key-here"
os.makedirs('/content/ndvi', exist_ok=True)
download_and_process('/content/ndvi', start_year=2024)

In [7]:
# rename to follow CYBench conventions

def doy_to_date(doy, year):
    # Based on
    # https://www.geeksforgeeks.org/python-convert-day-number-to-date-in-particular-year/

    # Pad on the left with 0's.
    # See https://docs.python.org/3/library/stdtypes.html. Look for str.rjust().
    if (isinstance(doy, int)):
        doy = str(doy)
    if (isinstance(year, int)):
        year = str(year)

    doy.rjust(3 + len(doy), '0')
    date_str = datetime.strptime(year + "-" + doy, "%Y-%j").strftime("%Y%m%d")

    return date_str

# example: /tmp/mod09.ndvi.global_0.05_degree.2024.001.c6.v1.tif
predictor_source = "MOD09CMG"
indicator = "ndvi"
dir_path = r'/content/ndvi'
files = [ f for f in os.listdir(dir_path) if f.endswith(".tif")]
for f in files:
    old_file = os.path.join(dir_path, f)
    try:
        year = f.split('.')[4]
        doy = f.split('.')[5]
        date_str = doy_to_date(doy, year)
        _, extension = os.path.splitext(f)
        new_name = "_".join([predictor_source, indicator, date_str]) + extension
        new_file = os.path.join(dir_path,  new_name)

        os.rename(old_file, new_file)

    except ValueError:
        continue